In [1]:
import os
import platform
import pandas as pd
import numpy as np
import tensorflow as tf
from zipfile import ZipFile
import time
import pyautogui
from PIL import Image
import pytesseract
import requests
from sklearn.metrics import mean_squared_error, mean_absolute_error
from math import sqrt
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, Dense, Concatenate, GlobalAveragePooling2D
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

# Adjustable parameters
latent_dim = 50  # Dimensionality of latent space
num_clients = 5  # Number of client models
epochs_per_client = 18  # Number of epochs for each client model
mu = 0.01  # Proximal term coefficient for FedProx
learning_rate = 0.0025  # Learning rate for optimization
batch_size = 32  # Batch size for training
image_size = (224, 224)  # Input image size for MobileNet

# Set Tesseract-OCR installation path for Windows
if platform.system() == "Windows":
    pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# Define paths to organized screenshots
organized_screenshots_path = "C:\\Users\\paray\\OneDrive\\Desktop\\bits new research\\dataset\\dataset\\organized_screenshots"
education_coursera_path = os.path.join(organized_screenshots_path, "Education", "Coursera")
education_programming_path = os.path.join(organized_screenshots_path, "Education", "Programming")
education_youtube_path = os.path.join(organized_screenshots_path, "Education", "YouTube")
entertainment_youtube_path = os.path.join(organized_screenshots_path, "Entertainment", "YouTube")
shopping_path = os.path.join(organized_screenshots_path, "Shopping")

# Load and preprocess screenshots
def load_and_preprocess_screenshots(folder_path, label, image_size=(224, 224)):
    images = []
    labels = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            img_path = os.path.join(folder_path, filename)
            try:
                img = Image.open(img_path).convert('RGB')  # Convert to RGB
                img = img.resize(image_size)
                img_array = np.array(img) / 255.0  # Normalize to [0, 1]
                print(f"Processed Image: {filename}, Shape: {img_array.shape}")  # Debugging
                images.append(img_array)
                labels.append(label)
            except Exception as e:
                print(f"Skipping {filename} due to error: {e}")
    return np.array(images), np.array(labels)

# Load data from all folders
X_coursera, y_coursera = load_and_preprocess_screenshots(education_coursera_path, label=0)
X_programming, y_programming = load_and_preprocess_screenshots(education_programming_path, label=1)
X_education_youtube, y_education_youtube = load_and_preprocess_screenshots(education_youtube_path, label=2)
X_entertainment_youtube, y_entertainment_youtube = load_and_preprocess_screenshots(entertainment_youtube_path, label=3)
X_shopping, y_shopping = load_and_preprocess_screenshots(shopping_path, label=4)

# Combine all data
X = np.concatenate([X_coursera, X_programming, X_education_youtube, X_entertainment_youtube, X_shopping])
y = np.concatenate([y_coursera, y_programming, y_education_youtube, y_entertainment_youtube, y_shopping])

# Convert labels to one-hot encoding
y = to_categorical(y, num_classes=5)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Feature extraction using MobileNet
base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze the base model

# Add a classification head
inputs = Input(shape=(224, 224, 3))
x = base_model(inputs, training=False)
x = GlobalAveragePooling2D()(x)
outputs = Dense(5, activation='softmax')(x)
model = Model(inputs, outputs)

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs_per_client, validation_data=(X_test, y_test))

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy}")

# Plot training history
plt.figure(figsize=(12, 6))
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

Processed Image: 0c65e1c1-c240-4f97-9e8d-c828274683d8_Education_Coursera_20250121_103729_4882dbf3-b596-4699-811c-0d2e7f80b773.png, Shape: (224, 224, 3)
Processed Image: 0c65e1c1-c240-4f97-9e8d-c828274683d8_Education_Coursera_20250121_103729_4882dbf3-b596-4699-811c-0d2e7f80b773_hog.png, Shape: (224, 224, 3)
Processed Image: 0c65e1c1-c240-4f97-9e8d-c828274683d8_Education_Coursera_20250121_103742_bbd1bf75-44ba-48ca-8a86-5cb7d2226cfe.png, Shape: (224, 224, 3)
Processed Image: 0c65e1c1-c240-4f97-9e8d-c828274683d8_Education_Coursera_20250121_103742_bbd1bf75-44ba-48ca-8a86-5cb7d2226cfe_hog.png, Shape: (224, 224, 3)
Processed Image: 127ddc4f-1fe5-414e-a46d-244d912f3bd8_Education_Coursera_20250120_185524_718813a1-cf73-4c2a-a2a9-7fcebea99e87.png, Shape: (224, 224, 3)
Processed Image: 127ddc4f-1fe5-414e-a46d-244d912f3bd8_Education_Coursera_20250120_185524_718813a1-cf73-4c2a-a2a9-7fcebea99e87_hog.png, Shape: (224, 224, 3)
Processed Image: 127ddc4f-1fe5-414e-a46d-244d912f3bd8_Education_Coursera_202

KeyboardInterrupt: 

In [4]:
import os
import numpy as np
from PIL import Image
import pyautogui
import time
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

# Load the model architecture
base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze the base model

# Add a classification head
inputs = Input(shape=(224, 224, 3))
x = base_model(inputs, training=False)
x = GlobalAveragePooling2D()(x)
outputs = Dense(5, activation='softmax')(x)
model = Model(inputs, outputs)

# Load the pre-trained weights
model.load_weights("C:\\Users\\paray\\OneDrive\\Desktop\\bits new research\\federated_model_weights.weights.h5")

# Function to preprocess an image
def preprocess_image(image, image_size=(224, 224)):
    """
    Preprocess an image for prediction.
    """
    img = image.resize(image_size)  # Resize to the input size of MobileNet
    img_array = np.array(img) / 255.0  # Normalize to [0, 1]
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    return img_array

# Class names
class_names = ["Coursera", "Programming", "Education YouTube", "Entertainment YouTube", "Shopping"]

# Continuous screenshot and prediction loop
try:
    while True:
        # Take a screenshot of the laptop screen
        screenshot = pyautogui.screenshot()

        # Preprocess the screenshot
        preprocessed_screenshot = preprocess_image(screenshot)

        # Predict the class probabilities
        predictions = model.predict(preprocessed_screenshot)

        # Get the predicted class label
        predicted_class = np.argmax(predictions, axis=1)

        # Map the predicted class index to the actual class name
        predicted_class_name = class_names[predicted_class[0]]

        # Print the predicted class
        print(f"The screenshot belongs to the class: {predicted_class_name}")

        # Add a delay between screenshots (e.g., 5 seconds)
        time.sleep(5)

except KeyboardInterrupt:
    print("Stopping continuous screenshot prediction...")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 447ms/step
The screenshot belongs to the class: Coursera
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
The screenshot belongs to the class: Coursera
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step
The screenshot belongs to the class: Coursera
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step
The screenshot belongs to the class: Coursera
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step
The screenshot belongs to the class: Entertainment YouTube
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 136ms/step
The screenshot belongs to the class: Coursera
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
The screenshot belongs to the class: Entertainment YouTube
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
The screenshot belongs to the class: Entertainment YouTube
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
The screenshot belongs to the class: Education YouTube
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
The screenshot belongs to the class: Entertainment YouTube
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
The screenshot belongs to the class: Entertainment YouTu